In [175]:
import spacy 
import pandas as pd 
from collections import Counter 
from string import punctuation
import json
nlp = spacy.load("en_core_web_lg")
movies = pd.read_csv('../datas/movie_keyword_importances.csv')

In [42]:
text = movies['overview'].iloc[0]
text

"Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences."

In [48]:

def get_hotwords(text):
    result = []
    pos_tag = ['PROPN', 'ADJ', 'NOUN']
    doc = nlp(text.lower())
    for token in doc:
        if(token.text in nlp.Defaults.stop_words or token.text in punctuation):
            continue
        if(token.pos_ in pos_tag):
            result.append(token.text)   
    return result

def count_percetages(text):
    output = get_hotwords(text)
    c = Counter(output)
    value_dict = {}
    for letter, count in c.most_common(7):
        value_dict[letter] =round((count/len(output))*100,2)
    return value_dict
    

In [53]:
count_percetages(text)
movies_1000 = movies.head(1000)
movies['movies_key'] = movies['overview'].apply(count_percetages)

,movieId,imdbId,tmdbId,Unnamed: 0,adult,belongs_to_collection,budget,genres,homepage,imdb_id,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,movies_key
0,1,114709,862.0,0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"['Animation', 'Comedy', 'Family']",http://toystory.disney.com/toy-story,tt0114709,...,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,"{'woody': 13.64, 'andy': 13.64, 'buzz': 13.64,..."
1,2,113497,8844.0,1,False,NaN,65000000,"['Adventure', 'Fantasy', 'Family']",NaN,tt0113497,...,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,"{'game': 11.54, 'alan': 7.69, 'siblings': 3.85..."
2,3,113228,15602.0,2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"['Romance', 'Comedy']",NaN,tt0113228,...,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,"{'max': 8.33, 'family': 4.17, 'wedding': 4.17,..."
3,4,114885,31357.0,3,False,NaN,16000000,"['Comedy', 'Drama', 'Romance']",NaN,tt0114885,...,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,"{'women': 6.25, 'breath': 6.25, 'elusive': 6.2..."
4,5,113041,11862.0,4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,['Comedy'],NaN,tt0113041,...,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,"{'george': 20.0, 'banks': 6.67, 'daughter': 6...."


In [61]:
movies['release_date'] = pd.to_datetime(movies['release_date'])

movies.to_csv('../datas/movie_keyword_importances.csv')

In [223]:
movies = pd.read_csv('../datas/movie_keyword_importances.csv')
movies['release_date'] = pd.to_datetime(movies['release_date'])
# clean some shits
movies.dropna(subset=['vote_count'], how='all', inplace=True)
movies = movies.loc[(movies['vote_count']) > (movies['vote_count'].mean())]

movies['year'] = movies['release_date'].dt.year


movies.to_csv('../datas/movie_keyword_importances.csv')

In [202]:

def select_date(year):
    movies['release_date']
    test = movies['overview'].where(movies['release_date'].dt.year == year)
    test = test.dropna()
    return test.values

def get_hotwords(test):
    result = []
    pos_tag = ['PROPN', 'ADJ', 'NOUN']
    for sentences in test:
        doc = nlp(str(sentences).lower())
        for token in doc:
            if(token.text in nlp.Defaults.stop_words or token.text in punctuation):
                continue
            if(token.pos_ in pos_tag):
                result.append(token.lemma_)   
    return result

def count_percetages(test, key):
    output = get_hotwords(test)
    c = Counter(output)
    value_dict = {}
    for letter, count in c.most_common(10):
        if letter in key.keys():
            continue 
        else:
            value_dict[letter] = round((count/len(c))*100,2)
    return value_dict

In [ ]:
content = select_date(2000)
count_percetages(content,key_word_all)


In [226]:
def best_movie_year(year, number):
    movies_test = movies.loc[movies['year'] == year]
    best = movies_test.nlargest(number ,['vote_average'])
    return best[['title','release_date']]

best_movie_year(1990, 5)


,title,release_date
1165,GoodFellas,1990-09-12
581,Dances with Wolves,1990-11-09
2981,Awakenings,1990-12-19
3371,Misery,1990-11-28
1195,Miller's Crossing,1990-09-21
